In [5]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [7]:
model_name = "mistralai/Mistral-7B-v0.1"

In [8]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
   model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [10]:
from transformers import TextGenerationPipeline

In [11]:
pipe  = TextGenerationPipeline(model = model, tokenizer = tokenizer)

In [16]:
prompt = '''I need to perform a named entity recognition task on a  text related with inclusion criteria in clinical trials.
The entities you need to recognize are: Condition, Value, Drug, Procedure, Measurement, Temporal, Observation, Person, Mood, Device and Pregnancy_considerations.
Particularly you have to produce the ouput in the BIO format. I will show you an example of the expected output.
Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
Output:
Patients O
with O
symptomatic O
CNS B-Condition
metastases I-Condition
or O
leptomeningeal B-Condition
involvement I-Condition

You can see that tokens without any entity are labeled as O, and the tokens that are part of an entity are labeled as B-<entity> or I-<entity> depending on if they are the beginning or the inside of the entity.
Please, just answer the question for this specific example and stop writting after that.
Input text: Patients who are eligible for coronary revascularization ( angioplasty or CABG )
Output:'''

In [37]:
print(pipe(prompt, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Patients O
who O
are O
eligible B-Condition
for B-Condition
coronary B-Condition
revascularization I-Condition
( I-Condition
angioplasty I-Condition
or I-Condition
CABG I-Condition
) I-Condition

Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
Output:
Patients O
with O
symptomatic O
CNS B-Condition
metastases I-Condition
or O
leptomeningeal B-Condition
involvement I-Condition

Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
Output:
Patients O
with O
symptomatic O
CNS B-Condition
metastases I-Condition
or O
leptomeningeal B-Condition
involvement I-Condition

Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
Output:
Patients O
with O
symptomatic O
CNS B-Condition
metastases I-Condition
or O
leptomeningeal B-Condition
involvement I-Condition

Input text: Patients with symptomatic CNS metastases or leptomeningeal involvement
Output:
Patients O
with O
symptomatic O
CNS B-Condition
metasta

In [25]:
prompt_new = '''I am working on a named entity recognition problem, in the context of clinical
trials eligibility criteria. I will show you the list of entities:
- Condition
- Value
- Drug
- Procedure
- Measurement
- Temporal
- Observation
- Person
- Mood
- Device

Your task consists in annotate the named entities in a given sentence in the format I will explain you.
I will explain you with some examples:

Example 1:
Input: Patients who have received prior chemotherapy for unresectable disease.
Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.

Example 2:
Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.
Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the investigator, would make the patient <Mood>inappropriate for <Observation>entry into this study</Observation>.

As you can see, in each example, the extracted entities are enclosed using the sintax: <ENT>text of the entity</ENT>.

Please now annotate as explained before the following sentence:

Input: Have had one prior platinum-based chemotherapy regimen for the treatment of primary disease.
Output:'''

In [26]:
print(pipe(prompt_new, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Have had one prior <Procedure>platinum-based chemotherapy regimen</Procedure> for the treatment of <Condition>primary disease</Condition>.






















































































































































































































































































































































































































































































In [29]:
# Examples with entities not in the examples provided to the model:
prompt_new = '''I am working on a named entity recognition problem, in the context of clinical
trials eligibility criteria. I will show you the list of entities:
- Condition
- Value
- Drug
- Procedure
- Measurement
- Temporal
- Observation
- Person
- Mood
- Device

Your task consists in annotate the named entities in a given sentence in the format I will explain you.
I will explain you with some examples:

Example 1:
Input: Patients who have received prior chemotherapy for unresectable disease.
Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.

Example 2:
Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.
Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the investigator, would make the patient <Mood>inappropriate for <Observation>entry into this study</Observation>.

As you can see, in each example, the extracted entities are enclosed using the sintax: <ENT>text of the entity</ENT>.

Please now annotate as explained before the following sentence:

Input: Must have had a treatment-free interval of greater than 6 months following response to platinum.
Output:'''

In [30]:
print(pipe(prompt_new, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Must have had a treatment-free interval of greater than <Measurement>6 months</Measurement> following response to <Drug>platinum</Drug>.




















































































































































































































































































































































































































































































In [35]:
# Examples with entities not in the examples provided to the model:
prompt_new = '''I am working on a named entity recognition problem, in the context of clinical
trials eligibility criteria. I will show you the list of entities:
- Condition
- Value
- Drug
- Procedure
- Measurement
- Temporal
- Observation
- Person
- Mood
- Device

Your task consists in annotate the named entities in a given sentence in the format I will explain you.
I will explain you with some examples:

Example 1:
Input: Patients who have received prior chemotherapy for unresectable disease.
Output: Patients who have received prior <Procedure>chemotherapy</Procedure> for <Condition>unresectable disease</Condition>.

Example 2:
Input: Patients with any other severe concurrent disease, which in the judgment of the investigator, would make the patient inappropriate for entry into this study.
Ouput: Patients with any other severe <Condition>concurrent disease</Condition>, which in the judgment of the investigator, would make the patient <Mood>inappropriate for <Observation>entry into this study</Observation>.

Example 3:
Input: Must have had a treatment-free interval of greater than 6 months following response to platinum.
Ouput: Must have had <Condition>a treatment-free interval</Condition> of <Temporal>greater than 6 months following response to platinum</Temporal>.

As you can see, in each example, the extracted entities are enclosed using the sintax: <ENT>text of the entity</ENT>. Take into account that sometimes it would be better to
keep the more general entity over specific ones.

Please now annotate as explained before the following sentence:

Input: At least 4 weeks since last surgery or radiation therapy.
Output:'''

In [36]:
print(pipe(prompt_new, max_new_tokens = 500, return_full_text = False, handle_long_generation = "hole")[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 At least 4 weeks since <Procedure>last surgery</Procedure> or <Procedure>radiation therapy</Procedure>.
























































































































































































































































































































































































































































































